In [3]:
import pandas as pd 
dataset=pd.read_json("../data/PLOS/PLOSArticles.json")

In [ ]:
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import csv
import pandas as pd

# Headers for making requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.137 Safari/537.36'
}

# Function to save data periodically by appending to CSV
def periodicSave(data):
    try:
        # Open file in append mode and write data
        with open("../data/PLOS/PLOSArticlesDetails.csv", mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(data)  # Append the new data row
        print("A Periodic Save is done.")
    except Exception as e:
        print(f"Error during periodic save: {e}")

# Function to collect article details from a given article URL
def collectArticleDetails(articleUrl):
    try:
        # Construct the full URL and initiate the request
        articleUrl = "https://journals.plos.org" + articleUrl
        print(f"Navigating to {articleUrl}")
        request = Request(articleUrl, headers=headers)

        # Open the URL and read the page content
        with urlopen(request) as response:
            page_source = response.read()

        # Parse the content with BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')
        
        # Find the download menu and the XML link
        downloadMenu = soup.find("div", class_="dload-menu")
        liElement = downloadMenu.find("a", id="downloadXml")
        
        # Construct the full XML link
        xmlLink = "https://journals.plos.org" + liElement.get("href")
        
        # Request the XML content
        request = Request(xmlLink, headers=headers)
        with urlopen(request) as response:
            xml_content = response.read()

        return xml_content
    except Exception as e:
        print(f"Error while fetching article details: {e}")
        return "Content is not available"

# Main loop to process the dataset and collect article details
def processDataset(dataset):
    for index, row in dataset.iterrows():
        # Collect the XML content for each article
        xml_content = collectArticleDetails(row["articleLink"])
                # Parse the XML content using BeautifulSoup
        soup_xml = BeautifulSoup(xml_content, 'xml')

        # Extract content between <abstract> and </abstract> tags
        abstract_tag = soup_xml.find("abstract")
        abstract_content = abstract_tag.get_text() if abstract_tag else ""

        # Extract content between <abbreviation> and </abbreviation> tags (if exists)
        abbreviation_tag = soup_xml.find("abbreviation")
        abbreviation_content = abbreviation_tag if abbreviation_tag else ""

        combined_content = abstract_content + "\n" + abbreviation_content
        combined_content=combined_content.strip()
        print(combined_content)
        # Add the XML content to the dataset
        dataset.loc[index, "xmlContent"] = xml_content

        # Save periodically after processing 500 entries
        if index % 100 == 0:
            data_to_save = dataset.iloc[:index + 1].values.tolist()
            periodicSave(data_to_save)




In [9]:
processDataset(dataset)

Navigating to https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0314876
Fibrotic interstitial lung diseases (ILDs) result from excessive deposition of extracellular matrix (ECM) proteins in the lung, causing irreversible damage to the lung architecture. Clinical management of ILDs differs depending on the diagnosis, but differentiation between subtypes can be difficult and better clinical biomarkers are needed. In this study, we use a 166-gene NanoString assay to investigate whether there are ILD subtype-specific transcripts in whole blood. We identified one transcript, killer cell lectin like receptor 1 (KLRF1), as differentially expressed between idiopathic pulmonary fibrosis (IPF) and systemic sclerosis-associated ILD (SSc-ILD), and identified two transcripts (VCAN, LTK) associated with IPF expression against other ILD subtypes. These findings were validated by examining their expression in ILD lung, with KLRF1 expression significantly higher in SSc-ILD compared to I

KeyboardInterrupt: 